In [2]:
import pandas as pd
import os
from tqdm import tqdm

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="gfdg")

In [3]:
df_supply=pd.read_csv("supply_lat_long.csv")
df_demand=pd.read_csv("demand_lat_long.csv")


In [15]:
import folium

my_map = folium.Map(location=[51.5074, 0.1278],
                    zoom_start = 5) # for UK 

df_supply.fillna(0,inplace=True)
for idx,row in df_supply.iterrows():
    # folium.Marker( location=[ (row["Latitude"]), (row["Longitude"]) ], fill_color='red', radius=1 ).add_to( my_map )
    folium.CircleMarker(location=[ (row["Latitude"]), (row["Longitude"]) ],
                        radius=1,
                        weight=5).add_to(my_map)

df_demand.fillna(0,inplace=True)
# for idx,row in df_demand.iterrows():
#     # folium.Marker( location=[ (row["Latitude"]), (row["Longitude"]) ], fill_color='red', radius=1 ).add_to( my_map )
#     folium.CircleMarker(location=[ (row["Latitude"]), (row["Longitude"]) ],
#                         radius=1,
#                         color='red',
#                         weight=5).add_to(my_map)
my_map

In [18]:
def dist_between_two_lat_lon(*args):
    from math import asin, cos, radians, sin, sqrt
    lat1, lat2, long1, long2 = map(radians, args)

    dist_lats = abs(lat2 - lat1) 
    dist_longs = abs(long2 - long1) 
    a = sin(dist_lats/2)**2 + cos(lat1) * cos(lat2) * sin(dist_longs/2)**2
    c = asin(sqrt(a)) * 2
    radius_earth = 6378 # the "Earth radius" R varies from 6356.752 km at the poles to 6378.137 km at the equator.
    return c * radius_earth

def find_closest_lat_lon(data, v):
    try:
        return min(data, key=lambda p: dist_between_two_lat_lon(v['Latitude'],p['Latitude'],v['Longitude'],p['Longitude']))
    except TypeError:
        print('Not a list or not a number.')

supply_dict=df_supply.to_dict("records")
demand_dict=df_demand.to_dict("records")



subset_of_demand=[]
supply_demand_map={}
for i in range(len(supply_dict)):

    a=find_closest_lat_lon(demand_dict, supply_dict[i])
    
    subset_of_demand.append(a)

    supply_demand_map[supply_dict[i]["Unnamed: 0"]]=a["Unnamed: 0"]

    demand_dict.remove(a)
supply_demand_map


{'sutherland_altnaharra': 'Falkirk',
 'cambridgeshire_monks-wood': 'Oxford',
 'greater-london_kenley-airfield': 'Croydon',
 'gwynedd_valley': 'East',
 'cumbria_shap': 'Eden',
 'bedfordshire_bedford': 'Bedford',
 'argyll-in-strathclyde-region_tiree': 'Aberdeen City',
 'powys-north_lake-vyrnwy': 'Isle of Anglesey',
 'nottinghamshire_nottingham-watnall': 'Broxtowe',
 'greater-london_kew-gardens': 'Ealing',
 'kent_east-malling': 'Maidstone',
 'nottinghamshire_sutton-bonington': 'North West Leicestershire',
 'somerset_liscombe': 'Isles of Scilly',
 'antrim_ballypatrick-forest': 'Scottish Borders',
 'ayrshire_auchincruive': 'Dumfries and Galloway',
 'cornwall_cardinham-bodmin': 'Cornwall',
 'inverness-shire_tulloch-bridge': 'Midlothian',
 'south-glamorgan_st-athan': 'Bridgend',
 'ayrshire_prestwick-gannet': 'Moray',
 'cheshire_rostherne': 'Trafford',
 'midlothian-in-lothian-region_edinburgh-gogarbank': 'Renfrewshire',
 'lincolnshire_waddington': 'Ribble Valley',
 'east-sussex_herstmonceux-we

In [19]:

for i in range(len(subset_of_demand)):
    # folium.Marker( location=[ (row["Latitude"]), (row["Longitude"]) ], fill_color='red', radius=1 ).add_to( my_map )
    folium.CircleMarker(location=[ (subset_of_demand[int(i)]["Latitude"]), (subset_of_demand[int(i)]["Longitude"]) ],
                        radius=1,
                        color="red",
                        weight=5).add_to(my_map)


# supply_df=pd.DataFrame(supply_dict)
# demand_df=pd.DataFrame(subset_of_demand)
# supply_df

my_map

In [38]:
supply_demand_map

df=pd.read_csv("./output/2015.csv")

place_lst=list(set(list(el.replace(".Solar","").replace(".Wind","").replace("-no-2","") for el in df.columns)))
place_lst.remove("ob_time")
for i in range(len(place_lst)):
    print(place_lst[i])
    supply_demand_map[place_lst[i]]

df=pd.DataFrame(supply_demand_map,index=["Demand"])
df.to_csv("IMPORT_SUPPLY_DEMAND_MAP.csv")

ross-and-cromarty_loch-glascarnoch
gwynedd_aberdaron
cambridgeshire_monks-wood
dumfriesshire_eskdalemuir
warwickshire_church-lawford
kent_east-malling
norfolk_norwich-airport
kirkcudbrightshire_dundrennan
durham_durham
sutherland_kinbrace-hatchery
cambridgeshire_wittering
sutherland_altnaharra
cheshire_rostherne
inverness-shire_aviemore
hertfordshire_rothamsted
bedfordshire_woburn
fermanagh_thomastown
midlothian-in-lothian-region_edinburgh-gogarbank
shetland_lerwick
greater-london_heathrow
antrim_ballypatrick-forest
cornwall_camborne
west-sussex_thorney-island
aberdeenshire_dyce
kent_manston
ross-and-cromarty_tain-range
greater-london_kenley-airfield
dorset_hurn
argyll-in-strathclyde-region_dunstaffnage
argyll-in-strathclyde-region_tiree
western-isles_stornoway-airport
east-sussex_herstmonceux-west-end
greater-london_kew-gardens
somerset_liscombe
cornwall_cardinham-bodmin
surrey_charlwood
moray-in-grampian-region_kinloss
powys-north_lake-vyrnwy
hereford-and-worcester_shobdon-airfield
a